In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from MdpSolverLibs import *
from EnvLibs import TrafficGenerator

In [ ]:
params = {
    'N_user': 4,
    'LEN_window': 10,
    'r_bar': 4,
    'B': 40,
    'M_list': [2,3],
    'randomSeed': 999,
    'alphaList': np.linspace(0, 1.0, 10),
    'N_aggregation': 4
}

lenWindow = 10
dataflow = "thumb_fr"
trafficGenerator = TrafficGenerator(params)
for taskName in ["Task0", "Task1", "Task2"]:
    with open(f'Results/TrafficData/trafficData_{taskName}_{dataflow}_LenWindow{lenWindow}.pkl', 'rb') as f:
        trafficData = pickle.load(f)
    trafficGenerator.registerDataset(trafficData['traffic'], train_ratio=0.7)
    print(f"Ts:{trafficData['Ts']}, Size: {trafficData['traffic'].shape}")

mdpFormulator = MdpFormulator(params, trafficGenerator.getM(mode="train"))
print(np.sum(trafficGenerator.getM(mode="train"), axis=1))
print(mdpFormulator.N_states)
print(mdpFormulator.N_actions)
print(mdpFormulator.N_states_original**mdpFormulator.N_user)
print(mdpFormulator.N_states_original**mdpFormulator.N_user*mdpFormulator.N_actions)

Ts:0.001, Size: (30817,)
Ts:0.001, Size: (36017,)
Ts:0.001, Size: (37230,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
256
320
14641
4685120


In [3]:
mdpFormulator.aggregateModel(approximate=True)

In [4]:
mdpKernel = mdpFormulator.getMdpKernel()
print(mdpKernel.rewardTable.shape)
print(mdpKernel.transitionTable.shape)
(V, policy) = mdpKernel.optimize_policy()

(256, 320)
(256, 256, 320)


In [8]:
with open(f'Results/MdpPolicy/mdpKernel_{dataflow}_LenWindow{lenWindow}.pkl', 'wb') as f:
    pickle.dump(mdpKernel, f)   
with open(f'Results/MdpPolicy/params_{dataflow}_LenWindow{lenWindow}.pkl', 'wb') as f:
    pickle.dump(mdpFormulator.params, f)
with open(f'Results/MdpPolicy/trafficGenerator_{dataflow}_LenWindow{lenWindow}.pkl', 'wb') as f:
    pickle.dump(trafficGenerator, f)
#print(mdpKernel.V)